In [0]:
#!/usr/bin/env python
# coding: utf-8

# In[5]:


#get_ipython().system('pip install tensorflow==1.14.0')


# In[3]:


from keras import layers 
import keras 
import keras.backend as K

width = 32 
height = 32 
channels = 1

input_layer = layers.Input(name='input', shape=(height, width, channels))
#1906.11632_gan_anomaly_detection.pdf
#Encoder
x = layers.Conv2D(32, (5,5), strides=(1,1), padding='same', name='conv_1', kernel_regularizer = 'l2')(input_layer)
x = layers.LeakyReLU(name='leaky_1')(x)

x = layers.Conv2D(64, (3,3), strides=(2,2), padding='same', name='conv_2', kernel_regularizer = 'l2')(x)
x = layers.BatchNormalization(name='norm_1')(x)
x = layers.LeakyReLU(name='leaky_2')(x)

x = layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='conv_3', kernel_regularizer = 'l2')(x)
x = layers.BatchNormalization(name='norm_2')(x)
x = layers.LeakyReLU(name='leaky_3')(x)

x = layers.Conv2D(256, (3,3), strides=(2,2), padding='same', name='conv_4', kernel_regularizer = 'l2')(x)
x = layers.BatchNormalization(name='norm_3')(x)
x = layers.LeakyReLU(name='leaky_4')(x)

x = layers.GlobalAveragePooling2D(name='g_encoder_output')(x)

g_e = keras.models.Model(inputs=input_layer, outputs=x)
#g_e.summary()

input_layer = layers.Input(name='input', shape=(height, width, channels))
#1906.11632_gan_anomaly_detection.pdf
#encoding + decoding
x = g_e(input_layer)

y = layers.Dense(width * width * 2, name='dense')(x) # 2 = 128 / 8 / 8
y = layers.Reshape((width//8, width//8, 128), name='de_reshape')(y)

y = layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', name='deconv_1', kernel_regularizer = 'l2')(y)
y = layers.LeakyReLU(name='de_leaky_1')(y)

y = layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', name='deconv_2', kernel_regularizer = 'l2')(y)
y = layers.LeakyReLU(name='de_leaky_2')(y)

y = layers.Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', name='deconv_3', kernel_regularizer = 'l2')(y)
y = layers.LeakyReLU(name='de_leaky_3')(y)

y = layers.Conv2DTranspose(channels, (1, 1), strides=(1,1), padding='same', name='decoder_deconv_output', kernel_regularizer = 'l2', activation='tanh')(y)

g = keras.models.Model(inputs=input_layer, outputs=y)
#g.summary()
g.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

input_layer = layers.Input(name='input', shape=(height, width, channels))
#1906.11632_gan_anomaly_detection.pdf
#second encoder
z = layers.Conv2D(32, (5,5), strides=(1,1), padding='same', name='encoder_conv_1', kernel_regularizer = 'l2')(input_layer)
z = layers.LeakyReLU()(z)

z = layers.Conv2D(64, (3,3), strides=(2,2), padding='same', name='encoder_conv_2', kernel_regularizer = 'l2')(z)
z = layers.BatchNormalization(name='encoder_norm_1')(z)
z = layers.LeakyReLU()(z)


z = layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='encoder_conv_3', kernel_regularizer = 'l2')(z)
z = layers.BatchNormalization(name='encoder_norm_2')(z)
z = layers.LeakyReLU()(z)

z = layers.Conv2D(256, (3,3), strides=(2,2), padding='same', name='conv_41', kernel_regularizer = 'l2')(z)
z = layers.BatchNormalization(name='encoder_norm_3')(z)
z = layers.LeakyReLU()(z)

z = layers.GlobalAveragePooling2D(name='encoder_output')(z)

encoder = keras.models.Model(input_layer, z)
#encoder.summary()

input_layer = layers.Input(name='input', shape=(height, width, channels))
#feature extraction layer
f = layers.Conv2D(32, (5,5), strides=(1,1), padding='same', name='f_conv_1', kernel_regularizer = 'l2')(input_layer)
f = layers.LeakyReLU(name='f_leaky_1')(f)

f = layers.Conv2D(64, (3,3), strides=(2,2), padding='same', name='f_conv_2', kernel_regularizer = 'l2')(f)
f = layers.BatchNormalization(name='f_norm_1')(f)
f = layers.LeakyReLU(name='f_leaky_2')(f)


f = layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='f_conv_3', kernel_regularizer = 'l2')(f)
f = layers.BatchNormalization(name='f_norm_2')(f)
f = layers.LeakyReLU(name='f_leaky_3')(f)


f = layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='f_conv_4', kernel_regularizer = 'l2')(f)
f = layers.BatchNormalization(name='f_norm_3')(f)
f = layers.LeakyReLU(name='feature_output')(f)

feature_extractor = keras.models.Model(input_layer, f)
#feature_extractor.summary()

class AdvLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AdvLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori_feature = feature_extractor(x[0])
        gan_feature = feature_extractor(x[1])
        return K.mean(K.square(ori_feature - K.mean(gan_feature, axis=0)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class CntLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CntLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.abs(ori - gan))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class EncLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(EncLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.square(g_e(ori) - encoder(gan)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)

# model for training
input_layer = layers.Input(name='input', shape=(height, width, channels))
gan = g(input_layer) # g(x)

adv_loss = AdvLoss(name='adv_loss')([input_layer, gan])
cnt_loss = CntLoss(name='cnt_loss')([input_layer, gan])
enc_loss = EncLoss(name='enc_loss')([input_layer, gan])

gan_trainer = keras.models.Model(input_layer, [adv_loss, cnt_loss, enc_loss])

# loss function
def loss(yt, yp):
    return yp

losses = {
    'adv_loss': loss,
    'cnt_loss': loss,
    'enc_loss': loss,
}

lossWeights = {'cnt_loss': 20.0, 'adv_loss': 1.0, 'enc_loss': 1.0}

# compile
gan_trainer.compile(optimizer = 'adam', loss=losses, loss_weights=lossWeights)
#gan_trainer.summary()

input_layer = layers.Input(name='input', shape=(height, width, channels))

f = feature_extractor(input_layer)

d = layers.GlobalAveragePooling2D(name='glb_avg')(f)
d = layers.Dense(1, activation='sigmoid', name='d_out')(d)

d = keras.models.Model(input_layer, d)
#d.summary()

d.compile(optimizer='adam', loss='binary_crossentropy')

from keras.datasets import mnist
from sklearn import preprocessing
import cv2
import numpy as np
import pickle

#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_ok = x_train[y_train == 1] # 6742 筆
#x_test = x_test[(y_test == 7) | (y_test == 1)] # 1135 筆 "1", 1028 筆 "7"
#y_test = y_test[(y_test == 7) | (y_test == 1)]

#training data
#from keras.preprocessing.image import ImageDataGenerator
#datagen = ImageDataGenerator(rescale=1./255)
#train_bg = datagen.flow_from_directory('/content/drive/My Drive/New Data/train', target_size=(32, 32), shuffle=True, class_mode='binary')
#x_ok, batchy = train_bg.next()
pickle_in = open("jj.h","rb")
X = pickle.load(pickle_in)
X_tr = []
X_trai = []
X_train = []
X_train1 = X['jet_image']
X_train = X_train1[:80000]
for i in range(0, len(X_tr)):
  X_trai.append(preprocessing.normalize(X_tr[i]))


#testing data
#datagen = ImageDataGenerator(rescale=1./255)
#train_sg = datagen.flow_from_directory('/content/drive/My Drive/New Data/test', target_size=(32, 32), class_mode='binary')
#x_test, batchyt = train_sg.next()
pickle_ins = open("ttx_had.h","rb")
Y = pickle.load(pickle_ins)
x_test1 = Y['jet_image']
x_test = []
x_te = []
x_tes = []
x_te = X_train1[80000:90000]
xyz = x_test1[:10000]
x_test = np.concatenate((x_te, xyz), axis=0)
for i in range(0, len(x_te)):
  x_tes.append(preprocessing.normalize(x_te[i]))
label = []
for i in range(10000):
  label.append(0)
for i in range(10000,20000):
  label.append(1)


#def reshape_x(x):
 #   new_x = np.empty((len(x), width, height))
  #  for i, e in enumerate(x):
   #     new_x[i] = cv2.resize(e, (width, height))
    #return np.expand_dims(new_x, axis=-1) / 127 - 1
  
#x_ok = reshape_x(x_ok)
#x_test = reshape_x(x_test)

niter = 70
bz = 128

def get_data_generator(data, batch_size=128):
    datalen = len(data)
    cnt = 0
    while True:
        idxes = np.arange(datalen)
        np.random.shuffle(idxes)
        cnt += 1
        for i in range(int(np.ceil(datalen/batch_size))):
            train_x = np.take(data, idxes[i*batch_size: (i+1) * batch_size], axis=0)
            y = np.ones(len(train_x))
            yield train_x, y

train_data_generator = get_data_generator(X_train, bz)

for i in range(niter):
    #break
    ### get batch x, y ###
    x, y = train_data_generator.__next__()
    x = np.reshape(x, [bz, 32, 32, 1])
        
    ### train disciminator ###
    d.trainable = True
        
    fake_x = g.predict(x)
    
    d_x = np.concatenate([x, fake_x], axis=0)
    d_y = np.concatenate([np.zeros(len(x)), np.ones(len(fake_x))], axis=0)

    d_loss = d.train_on_batch(d_x, d_y)

    ### train generator ###
    
    #d.trainable = False        
    #g_loss = gan_trainer.train_on_batch(x, y)
    #g_loss = g.train_on_batch(x, y)
    
    #if i % 10 == 0:
    print(f'niter: {i+1}, g_loss: , d_loss: {d_loss}')
d.trainable = False
X_train = np.reshape(X_train, [len(X_train), 32, 32, 1])
g_loss = g.fit(X_train, X_train, batch_size=64, epochs=1, shuffle=True)
print (g_loss)

x_test = np.reshape(x_test, [20000, 32, 32, 1])
encoded = g_e.predict(x_test)
gan_x = g.predict(x_test)
encoded_gan = encoder.predict(gan_x)
score = []
sc = []
sc = np.absolute(encoded - encoded_gan)
####################################################
#-----------------------------------------------------#
#score= np.sum(sc,axis=1)
#score=score/(np.max(score)-np.min(score))
#----------------------------------------------------#
score_= np.sqrt(np.sum(sc**2,axis=1))
score=(score_-np.min(score_))/(np.max(score_)-np.min(score_))
print (np.min(score),np.max(score))
###################################################

from sklearn.metrics import roc_curve, auc
#p = g.predict(x_test)
lr_fpr, lr_tpr, _ = roc_curve(label, score)

from matplotlib import pyplot as plt
#from pylab import rcParams
#rcParams['figure.figsize'] = 14, 5
#plt.scatter(range(len(x_test)), score, c=['skyblue' if x == 0 else 'pink' for x in batchyt])


plt.plot(lr_fpr, lr_tpr, marker='.', label='Ganomaly')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()
print (auc(lr_fpr, lr_tpr))

